## Application 1: The Union Wage Premium

We estimate the effect of union membership on wages (i.e., the union wage premium) controlling for 10 other variables that may be correlated with hourly wages.

The coefficient on union, 11.1, means that union members earn about 11 percent more than nonunion members. This is on the low side of conventional estimates, which center around a 15 percent premium (Hirsch 2004).

In [1]:
import pandas as pd

df = pd.read_stata('nlsw88.dta', index_col="idcode")
df.head()

,age,race,married,never_married,grade,collgrad,south,smsa,c_city,industry,occupation,union,wage,hours,ttl_exp,tenure
idcode,,,,,,,,,,,,,,,,
1,37,black,single,0,12.0,not college grad,0,SMSA,0,Transport/Comm/Utility,Operatives,union,11.739125,48.0,10.333334,5.333333
2,37,black,single,0,12.0,not college grad,0,SMSA,1,Manufacturing,Craftsmen,union,6.400963,40.0,13.621795,5.250000
3,42,black,single,1,12.0,not college grad,0,SMSA,1,Manufacturing,Sales,NaN,5.016723,40.0,17.730770,1.250000
4,43,white,married,0,17.0,college grad,0,SMSA,0,Professional Services,Other,union,9.033813,42.0,13.211537,1.750000
6,42,white,married,0,12.0,not college grad,0,SMSA,0,Manufacturing,Operatives,nonunion,8.083731,48.0,17.820513,17.750000


In [2]:
df.describe()

,age,never_married,grade,south,c_city,wage,hours,ttl_exp,tenure
count,2246.000000,2246.000000,2244.000000,2246.000000,2246.000000,2246.000000,2242.000000,2246.000000,2231.000000
mean,39.153161,0.104185,13.098930,0.419412,0.291630,7.766931,37.218109,12.534982,5.977849
std,3.060002,0.305569,2.521246,0.493573,0.454614,5.755528,10.509135,4.610212,5.510326
min,34.000000,0.000000,0.000000,0.000000,0.000000,1.004952,1.000000,0.115385,0.000000
25%,36.000000,0.000000,12.000000,0.000000,0.000000,4.261271,35.000000,9.213141,1.583333
50%,39.000000,0.000000,12.000000,0.000000,0.000000,6.272270,40.000000,13.125000,3.833333
75%,42.000000,0.000000,15.000000,1.000000,1.000000,9.593397,40.000000,15.979167,9.333333
max,46.000000,1.000000,18.000000,1.000000,1.000000,40.746590,80.000000,28.884615,25.916666


In [3]:
indep_vars = ['union', 'hours', 'age', 'grade', 'collgrad', 'south', 'smsa', 'c_city', 'ttl_exp', 'tenure']
dep_var = ['wage']
df = df[indep_vars + dep_var]
df.head()

,union,hours,age,grade,collgrad,south,smsa,c_city,ttl_exp,tenure,wage
idcode,,,,,,,,,,,
1,union,48.0,37,12.0,not college grad,0,SMSA,0,10.333334,5.333333,11.739125
2,union,40.0,37,12.0,not college grad,0,SMSA,1,13.621795,5.250000,6.400963
3,NaN,40.0,42,12.0,not college grad,0,SMSA,1,17.730770,1.250000,5.016723
4,union,42.0,43,17.0,college grad,0,SMSA,0,13.211537,1.750000,9.033813
6,nonunion,48.0,42,12.0,not college grad,0,SMSA,0,17.820513,17.750000,8.083731


In [4]:
print(len(df))
df.isnull().sum()

2246


union       368
hours         4
age           0
grade         2
collgrad      0
south         0
smsa          0
c_city        0
ttl_exp       0
tenure       15
wage          0
dtype: int64

In [15]:
import altair as alt

cat_cols = ["union", "collgrad", "south", "smsa", "c_city"]
alt.data_transformers.disable_max_rows()

def create_df_col_hist_chart(df, y_col):
    charts = []
    columns = df.columns
    for c in columns:
        chart = alt.Chart(df).mark_bar().encode(
            x=alt.X(f'{c}:N') if c in cat_cols else alt.X(f'{c}', bin=alt.BinParams(maxbins = 50)),
            y='count()',
            color=y_col
        )
        charts.append(chart)
    chart = alt.hconcat()

    for temp_chart in charts:
        chart |= temp_chart
    return chart

create_df_col_hist_chart(df, 'wage:Q')

alt.HConcatChart(...)

In [16]:
df.to_csv('union_wage.csv')

In [10]:
import altair as alt
chart = alt.Chart(df).mark_bar().transform_calculate(
            x = 'log(datum.wage) * 100'
        ).encode(
            x=alt.X(f'x', bin=alt.BinParams(maxbins = 50)),
            y='count()',
            color='union'
        )
chart

ValueError: x encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.Chart(...)